- loc 및 iloc 접근자는 대상으로 지정하려는 행/열의 인덱스 레이블과 위치를 알고 있을 때 사용 가능
-> 식별자가 아닌 조건이나 기준으로 행을 지정하고 싶을 때의 필터링 방법

# 데이터셋과 메모리 최적화
- 데이터셋을 가져올 때마다 각 열이 데이터를 가장 최적의 유형으로 저장하는지 확인하기
  - 최적의 유형: 가장 적은 메모리를 사용하거나 가장 많은 유틸리티를 제공하는 데이터 유형
  - ex. 정수->부동소수점이 아닌 정수형, 날짜/시간 유형->문자열 대신 특화된 메서드를 제공하는 유형

In [2]:
import pandas as pd

In [3]:
pd.read_csv("employees.csv")

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,8/6/93,NaN,True,Marketing
1,Thomas,Male,3/31/96,61933.0,True,NaN
2,Maria,Female,NaN,130590.0,False,Finance
3,Jerry,NaN,3/4/05,138705.0,True,Finance
4,Larry,Male,1/24/98,101004.0,True,IT
...,...,...,...,...,...,...
996,Phillip,Male,1/31/84,42392.0,False,Finance
997,Russell,Male,5/20/13,96914.0,False,Product
998,Larry,Male,4/20/13,60500.0,False,Business Dev
999,Albert,Male,5/15/12,129949.0,True,Sales


In [4]:
employees = pd.read_csv("employees.csv", parse_dates = ["Start Date"])

In [5]:
#info 메서드: 각 열의 데이터 유형, 결측값의 개수를 목록으로 확인
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    object        
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      999 non-null    float64       
 4   Mgmt        933 non-null    object        
 5   Team        957 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 47.0+ KB


## astype 메서드를 사용하여 데이터 유형 변환
- Mgmt 열의 값을 문자열로 가져옴 -> 불리언 데이터 유형으로 변환 가능
- astype 메서드: Series의 값을 다른 데이터 유형으로 변환

In [6]:
employees["Mgmt"].astype(bool) #결측값을 True로 가져옴

0        True
1        True
2       False
3        True
4        True
        ...  
996     False
997     False
998     False
999      True
1000     True
Name: Mgmt, Length: 1001, dtype: bool

- DataFrame의 열 업데이트 ~= 딕셔너리에서 키-값 쌍을 설정하는 것
  - 지정된 이름의 열이 존재하는 경우, 판다스는 이를 새 Series로 덮어씀
  - 지정된 이름의 열이 존재하지 않는 경우, 판다스는 새 Series를 생성하고 DataFrame 오른쪽에 추가. 라이브러리는 공통 인덱스 레이블로 Series와 DF의 행을 연결.

In [7]:
employees["Mgmt"] = employees["Mgmt"].astype(bool)

In [8]:
employees.tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev
999,Albert,Male,2012-05-15,129949.0,True,Sales
1000,NaN,NaN,NaT,NaN,True,NaN


In [9]:
#메모리 사용량 비교
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    object        
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      999 non-null    float64       
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(3)
memory usage: 40.2+ KB


In [10]:
#employees에서 Salary열의 값엔 NaN이 있기 때문에 정수형이 아닌 부동소수점으로 저장
#astype으로 강제 변환 시 IntCastingNaNError 발생
employees["Salary"].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [11]:
employees["Salary"] = employees["Salary"].fillna(0).astype(int)

In [12]:
#범주형 데이터 유형. 전체 개수에 비해 소수의 고유값으로 구성된 열에 적용.
employees.nunique()

First Name    200
Gender          2
Start Date    971
Salary        995
Mgmt            2
Team           10
dtype: int64

In [13]:
#소수의 고유값을 가진 Gender 채택
employees["Gender"].astype("category")

0         Male
1         Male
2       Female
3          NaN
4         Male
         ...  
996       Male
997       Male
998       Male
999       Male
1000       NaN
Name: Gender, Length: 1001, dtype: category
Categories (2, object): ['Female', 'Male']

In [14]:
employees["Gender"] = employees["Gender"].astype("category")

In [15]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    category      
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      1001 non-null   int32         
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    object        
dtypes: bool(1), category(1), datetime64[ns](1), int32(1), object(2)
memory usage: 29.6+ KB


In [16]:
#소수의 고유값을 가진 Team열 채택
employees["Team"] = employees["Team"].astype("category")
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    category      
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      1001 non-null   int32         
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    category      
dtypes: bool(1), category(2), datetime64[ns](1), int32(1), object(1)
memory usage: 23.1+ KB


# 단일 조건으로 필터링
- 데이터의 하위 집합(어떤 종류의 조건에 맞는 데이터셋의 일부)을 추출

In [17]:
#등호 연산자를 활용한 구문(Series == value) -> 불리언 Series를 반환
#판다스는 Series 자체가 아닌 각 Series 값을 지정된 문자열과 비교 가능
employees["First Name"] == "Maria"

0       False
1       False
2        True
3       False
4       False
        ...  
996     False
997     False
998     False
999     False
1000    False
Name: First Name, Length: 1001, dtype: bool

In [18]:
#필터링-1
employees[employees["First Name"] == "Maria"]

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance
198,Maria,Female,1990-12-27,36067,True,Product
815,Maria,NaN,1986-01-18,106562,False,HR
844,Maria,NaN,1985-06-19,148857,False,Legal
936,Maria,Female,2003-03-14,96250,False,Business Dev
984,Maria,Female,2011-10-15,43455,False,Engineering


In [19]:
#필터링-2
marias = employees["First Name"] == "Maria"
employees[marias]

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance
198,Maria,Female,1990-12-27,36067,True,Product
815,Maria,NaN,1986-01-18,106562,False,HR
844,Maria,NaN,1985-06-19,148857,False,Legal
936,Maria,Female,2003-03-14,96250,False,Business Dev
984,Maria,Female,2011-10-15,43455,False,Engineering


In [20]:
#부등호 연산자를 활용한 구문
employees[employees["Team"] != "Finance"]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
4,Larry,Male,1998-01-24,101004,True,IT
5,Dennis,Male,1987-04-18,115163,False,Legal
6,Ruby,Female,1987-08-17,65476,True,Product
...,...,...,...,...,...,...
995,Henry,NaN,2014-11-23,132483,False,Distribution
997,Russell,Male,2013-05-20,96914,False,Product
998,Larry,Male,2013-04-20,60500,False,Business Dev
999,Albert,Male,2012-05-15,129949,True,Sales


In [21]:
#불리언 값을 가진 열은 등호/부등호 연산자를 굳이 사용할 필요가 없음
employees[employees["Mgmt"]].head() #employees[employees["Mgmt"] == True]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT
6,Ruby,Female,1987-08-17,65476,True,Product


In [22]:
high_earners = employees["Salary"] > 100000 #해당 조건을 만족하는 불리언 Series를 생성
high_earners.head()

0    False
1    False
2     True
3     True
4     True
Name: Salary, dtype: bool

In [23]:
employees[high_earners].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT
5,Dennis,Male,1987-04-18,115163,False,Legal
9,Frances,Female,2002-08-08,139852,True,Business Dev


# 다중 조건으로 필터링
- 2개의 독립적인 불리언 Series를 만든 다음 두 Series 사이에 적용해야하는 논리적 기준을 선언하여 다중 조건 필터링 가능
## AND 조건

In [24]:
is_female = employees["Gender"] == "Female"
in_biz_dev = employees["Team"] == "Business Dev"
employees[is_female & in_biz_dev].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
9,Frances,Female,2002-08-08,139852,True,Business Dev
33,Jean,Female,1993-12-18,119082,False,Business Dev
36,Rachel,Female,2009-02-16,142032,False,Business Dev
38,Stephanie,Female,1986-09-13,36844,True,Business Dev
61,Denise,Female,2001-11-06,106862,False,Business Dev


In [25]:
#원하는 개수만큼 조건을 포함 가능
is_manager = employees["Mgmt"]
employees[is_female & in_biz_dev & is_manager].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
9,Frances,Female,2002-08-08,139852,True,Business Dev
38,Stephanie,Female,1986-09-13,36844,True,Business Dev
66,Nancy,Female,2012-12-15,125250,True,Business Dev
92,Linda,Female,2000-05-25,119009,True,Business Dev
111,Bonnie,Female,1999-12-17,42153,True,Business Dev


## OR 조건

In [26]:
earning_below_40k = employees["Salary"] < 40000
started_after_2015 = employees["Start Date"] > "2015-01-01"
employees[earning_below_40k | started_after_2015].tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
958,Gloria,Female,1987-10-24,39833,False,Engineering
964,Bruce,Male,1980-05-07,35802,True,Sales
967,Thomas,Male,2016-03-12,105681,False,Engineering
989,Justin,NaN,1991-02-10,38344,False,Legal
1000,NaN,NaN,NaT,0,True,NaN


## ~ 기호로 반전

In [27]:
my_series = pd.Series([True, False, True])
my_series

0     True
1    False
2     True
dtype: bool

In [28]:
~my_series

0    False
1     True
2    False
dtype: bool

In [29]:
#연봉 100000달러 미만인 직원을 식별 -1
employees[employees["Salary"] < 100000].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
6,Ruby,Female,1987-08-17,65476,True,Product
7,NaN,Female,2015-07-20,45906,True,Finance
8,Angela,Female,2005-11-22,95570,True,Engineering


In [30]:
#연봉 100000달러 미만인 직원을 식별 -2
employees[~(employees["Salary"] >= 100000)].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
6,Ruby,Female,1987-08-17,65476,True,Product
7,NaN,Female,2015-07-20,45906,True,Finance
8,Angela,Female,2005-11-22,95570,True,Engineering


## 불리언 메서드
- 등호(==): eq
- 부등호(!=): ne
- 작음(<): lt
- 작거나 같음(<=): le
- 큼(>): gt
- 크거나 같음(>=): ge

# 조건별 필터링
- 복잡한 유형의 추출용 불리언 Series를 생성할 때 활용할 수 있는 헬퍼 메서드를 제공

## isin 메서드
- '==' 조건이 여러 개일 경우의 확장성을 지원.
- 반복 가능한 요소(리스트, 튜플, Series 등)를 인수로 받아 불리언 Series를 반환하는 메서드

In [32]:
sales = employees["Team"] == "Sales"
legal = employees["Team"] == "Legal"
mktg = employees["Team"] == "Marketing"
employees[sales | legal | mktg].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
5,Dennis,Male,1987-04-18,115163,False,Legal
11,Julie,Female,1997-10-26,102508,True,Legal
13,Gary,Male,2008-01-27,109831,False,Sales
20,Lois,NaN,1995-04-22,64714,True,Legal


In [33]:
all_star_teams = ["Sales", "Legal", "Marketing"]
on_all_star_teams = employees["Team"].isin(all_star_teams)
employees[on_all_star_teams].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
5,Dennis,Male,1987-04-18,115163,False,Legal
11,Julie,Female,1997-10-26,102508,True,Legal
13,Gary,Male,2008-01-27,109831,False,Sales
20,Lois,NaN,1995-04-22,64714,True,Legal


## between 메서드
- 숫자나 날짜를 다룰 때, 범위에 속하는 값을 구하는 경우
  1. 하한, 상한을 나타내는 2개의 불리언 Series를 생성 후 & 연산자를 사용
  2. between 메서드: 하한과 상한을 인수로 받아 불리언 Series를 반환

In [35]:
#1번 방법
higher_than_80 = employees["Salary"] >= 80000
lower_than_90 = employees["Salary"] < 90000
employees[higher_than_80 & lower_than_90].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
19,Donna,Female,2010-07-22,81014,False,Product
31,Joyce,NaN,2005-02-20,88657,False,Product
35,Theresa,Female,2006-10-10,85182,False,Sales
45,Roger,Male,1980-04-17,88010,True,Sales
54,Sara,Female,2007-08-15,83677,False,Engineering


In [36]:
between_80k_and_90k = employees["Salary"].between(80000, 90000)
employees[between_80k_and_90k].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
19,Donna,Female,2010-07-22,81014,False,Product
31,Joyce,NaN,2005-02-20,88657,False,Product
35,Theresa,Female,2006-10-10,85182,False,Sales
45,Roger,Male,1980-04-17,88010,True,Sales
54,Sara,Female,2007-08-15,83677,False,Engineering


- between 메서드는 다른 데이터 유형의 열에도 적용 가능
  - ex. 날짜/시간 열 -> 문자열: 첫번째/두번째 인수에 대한 키워드 매개변수: left, right

In [37]:
eighties_folk = employees["Start Date"].between(
    left = "1980-01-01",
    right = "1990-01-01"
)
employees[eighties_folk].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
5,Dennis,Male,1987-04-18,115163,False,Legal
6,Ruby,Female,1987-08-17,65476,True,Product
10,Louise,Female,1980-08-12,63241,True,NaN
12,Brandon,Male,1980-12-01,112807,True,HR
17,Shawn,Male,1986-12-07,111737,False,Product


In [39]:
name_starts_with_r = employees["First Name"].between("R", "S")
employees[name_starts_with_r].head()
#문자 및 문자열 작업 시 대소문자 구분!

,First Name,Gender,Start Date,Salary,Mgmt,Team
6,Ruby,Female,1987-08-17,65476,True,Product
36,Rachel,Female,2009-02-16,142032,False,Business Dev
45,Roger,Male,1980-04-17,88010,True,Sales
67,Rachel,Female,1999-08-16,51178,True,Finance
78,Robin,Female,1983-06-04,114797,True,Sales


## isnull과 notnull 메서드
- isnull 메서드는 행의 값이 누락되면 True값을 가지는 불리언 Series를 반환
- 판다스는 NaT와 None값도 null로 간주

In [41]:
employees["Team"].isnull().head() #isnull() 사용-명시적 

0    False
1     True
2    False
3    False
4    False
Name: Team, dtype: bool

In [42]:
employees["Team"].notnull().head() #notnull() 사용-명시적

0     True
1    False
2     True
3     True
4     True
Name: Team, dtype: bool

In [43]:
~(employees["Team"].isnull()).head() #~isnull() 사용

0     True
1    False
2     True
3     True
4     True
Name: Team, dtype: bool

In [44]:
no_team = employees["Team"].isnull()
employees[no_team].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
1,Thomas,Male,1996-03-31,61933,True,NaN
10,Louise,Female,1980-08-12,63241,True,NaN
23,NaN,Male,2012-06-14,125792,True,NaN
32,NaN,Male,1998-08-21,122340,True,NaN
91,James,NaN,2005-01-26,128771,False,NaN


In [45]:
has_name = employees["First Name"].notnull()
employees[has_name].tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
995,Henry,NaN,2014-11-23,132483,False,Distribution
996,Phillip,Male,1984-01-31,42392,False,Finance
997,Russell,Male,2013-05-20,96914,False,Product
998,Larry,Male,2013-04-20,60500,False,Business Dev
999,Albert,Male,2012-05-15,129949,True,Sales


## null 값 다루기

In [47]:
employees = pd.read_csv("employees.csv", parse_dates = ["Start Date"])
employees

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,NaN,True,Marketing
1,Thomas,Male,1996-03-31,61933.0,True,NaN
2,Maria,Female,NaT,130590.0,False,Finance
3,Jerry,NaN,2005-03-04,138705.0,True,Finance
4,Larry,Male,1998-01-24,101004.0,True,IT
...,...,...,...,...,...,...
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev
999,Albert,Male,2012-05-15,129949.0,True,Sales


In [50]:
#dropna 메서드는 몇 개의 열 값이 결측값인지 상관없이 NaN이 하나라도 있는 DataFrame행을 제거
#how의 기본 인수: any
employees.dropna()
#employees.dropna(how = "any")

,First Name,Gender,Start Date,Salary,Mgmt,Team
4,Larry,Male,1998-01-24,101004.0,True,IT
5,Dennis,Male,1987-04-18,115163.0,False,Legal
6,Ruby,Female,1987-08-17,65476.0,True,Product
8,Angela,Female,2005-11-22,95570.0,True,Engineering
9,Frances,Female,2002-08-08,139852.0,True,Business Dev
...,...,...,...,...,...,...
994,George,Male,2013-06-21,98874.0,True,Marketing
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev


In [51]:
#dropna 메서드: 모든 값이 누락된 행만 제거->how 매개변수에 인수로 all을 전달
employees.dropna(how = "all").tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
995,Henry,NaN,2014-11-23,132483.0,False,Distribution
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev
999,Albert,Male,2012-05-15,129949.0,True,Sales


In [52]:
#dropna 메서드: 특정 열에 결측값이 있는 행을 지정하여 제거->subset 매개변수에 해당하는 열들의 리스트를 전달
employees.dropna(subset = ["Gender"]).tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
994,George,Male,2013-06-21,98874.0,True,Marketing
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev
999,Albert,Male,2012-05-15,129949.0,True,Sales


In [54]:
#dropna 메서드: thresh 매개변수는 판다스가 행을 유지하는 조건으로 
#최소 몇 개의 null이 아닌 값을 가져야하는지 결정하는 임계값
employees.dropna(thresh = 4).head()
#how와 thresh를 동시에 쓰면 에러남.

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,NaN,True,Marketing
1,Thomas,Male,1996-03-31,61933.0,True,NaN
2,Maria,Female,NaT,130590.0,False,Finance
3,Jerry,NaN,2005-03-04,138705.0,True,Finance
4,Larry,Male,1998-01-24,101004.0,True,IT


# 중복 처리
## duplicated 메서드